### Import and configure
We import all necessary libraries, and configure the selenium chromedriver. If you run this on a PC, you will need to change `./chromedriver/chromedriver_mac` to `./chromedriver/chromedriver.exe`.
We also set matplotlib to use a custom style.

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome('./chromedriver/chromedriver_mac', options=options)
driver.implicitly_wait(5)
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import style
# my matplotlib style, included in this repo.
style.use('misc/elip12.mplstyle')
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline

### Fetch initial data and set global variables
We read the tables from our first data source [1].
We also instantiate a dictionary of our custom matplotlib colors.

In [ ]:
rsc1 = pd.read_html('https://link.springer.com/article/10.1007/s10584-014-1169-1#Sec11')
d = {'g': '#60d515', 'r': '#d22b10', 'b': '#38d8dc', 'y': '#e0cc05',
    'o': '#eb860d', 'm': '#b113ef'}

### Plot emissions by diet type
We index the relevant table from the pandas read_html query (since read_html returns all tables in the document).
We reindex the dataframe for ease of plotting, and munge the data as necessary. We drop the fish-eaters
column since it is not relevant to our questions.
We make a bar plot from the dataframe.


In [ ]:
# extract dataframe
emissions_by_diet_type_cross_gender = rsc1[2].copy()

# create new multiindex from lists
outer = ['Men', 'Men', 'Men', 'Women', 'Women', 'Women', 'Adjusted for age and sex', 'Adjusted for age and sex']
inner = ['N', 'Mean dietary GHG emissions (kgCO2e)', 'SD', 'N',
         'Mean dietary GHG emissions (kgCO2e)', 'SD', 'Mean dietary GHG emissions (kgCO2e)', '95 % CIs']
index = pd.MultiIndex.from_arrays([outer, inner], names=['Gender', ' Emissions'])

# rename df columns
cols = {
    'Unnamed: 0': 'Diet Type',
}
emissions_by_diet_type_cross_gender.rename(columns=cols, inplace=True)

# reindex and wrangle data into form we want
emissions_by_diet_type_cross_gender.set_index(emissions_by_diet_type_cross_gender['Diet Type'],  inplace=True)
emissions_by_diet_type_cross_gender.drop('Diet Type', axis=1, inplace=True)
emissions_by_diet_type_cross_gender = emissions_by_diet_type_cross_gender.iloc[1:]
emissions_by_diet_type_cross_gender = pd.DataFrame(emissions_by_diet_type_cross_gender.values, columns=index, index=emissions_by_diet_type_cross_gender.index)
index_list = emissions_by_diet_type_cross_gender.index.values

# slice dataframe to get relvant information to plot
idx = pd.IndexSlice
series = emissions_by_diet_type_cross_gender.loc[:, idx[['Adjusted for age and sex'],['Mean dietary GHG emissions (kgCO2e)']]].astype(np.float64)

# remove unnecessary and useless data
series = series.drop('Fish-eaters')
series = series.dropna()

# grams consumed per day for each of three vegan meals.
# This lets us convert emissions per gram of diet to emissions per day of diet
d3 = 81 + 96 + 328
d2 = 54 + 401 + 128
d1 = 87 + 107 + 358
grams_per_day_v = (d1 + d2 + d3) / 3
hundred_grams_per_day_v = grams_per_day_v / 100
series.loc['Vegetarians'] *= hundred_grams_per_day_v
series.loc['Vegans'] *= hundred_grams_per_day_v

# number manually retrieved from data source 2
grams_per_day_m = 786 * 0.65 # normalized to remove alcohol and water. alcohol and water combined make up 35% of daily intake by mass
hundred_grams_per_day_m = grams_per_day_m / 100
series.iloc[[0,1,2]] *= hundred_grams_per_day_m

index = series.index.values
index[0] = 'High meat-eaters\n(> 100 g/day)'
index[1] = 'Medium meat-eaters\n(50-99 g/day)'
index[2] = 'Low meat-eaters\n(< 50 g/day)'
series = series.reindex(index)
series.to_csv('data/emissions_by_diet_type_processed.csv')

# create and format bar plot
plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.facecolor'] = '#252525'
fig = plt.figure()
props = dict(
    width=0.3,
    title='UK emissions by Diet Type, Adjusted for Age and Sex, Mormalizing CH4 and NO2 to Equivalent kg CO2',
    legend=False)
colors = [['#ea480e', '#ea580e', '#ea680e', '#ea780e', '#ea880e']]
ax = series.plot.bar(zorder=3, **props, color=colors)
ax.set_ylabel('kg diet-caused CO2 eqv per day', color='#CCCCCC')
ax.set_xlabel('\nDiet Type', color='#CCCCCC')
for tick in ax.get_xticklabels():
    tick.set_rotation(0)
plt.savefig('plots/emissions_by_diet_type.png', dpi=150)

### Plot emissions by food category
We start with a table that has emissions data (in the form of kilograms of CO2-equivalent greenhouse gasses per 100g food) for many different food types [1]. We remove unnecessary foods and reorder the columns. We add cheese from a different data source [2]. Note that you need to be on the campus wifi or use the VPN to access [2]. We create a bar plot, where bars are grouped by food group, and color-coded with a custom legend.

In [ ]:
# extract dataframe
emissions_by_food_category = rsc1[3]
df = emissions_by_food_category

# other table had no data for cheese, so I incorporated it from another similar paper [2]
url = 'https://www.sciencedirect.com/science/article/pii/S0921800912000043?via%3Dihub'
driver.get(url)
cheese = pd.read_html(
    driver.find_element_by_id("t0035").get_attribute('outerHTML'))[0]
cheese = cheese[['Low threshold','High threshold']]
cheese_mean = cheese.iloc[18:20].values.mean() / 1000
dfc = pd.DataFrame([['Cheese', cheese_mean]], columns=df.columns)
df = df.append(dfc, ignore_index=True)

df = df.iloc[[1, 12, 47, 62, 67, 26, 13, 22, 45, 94, 0, 92, 8, 9, 40, 48, 66, 72, 80, 83, 87, 88, 89, 90, 24, 6, 32, 53, 55, 59, 64, 65, 77]]
df = df.drop_duplicates()
df = df.set_index('Food category')
df /= 10
df = df.round(2)

df.to_csv('data/food_emissions.csv')

colors = 'rrrrrrmmmmbbbbbbbbbgggggggggggggy'
colors = [d[c] for c in list(colors)]


# plot and format
plt.rcParams['figure.figsize'] = [20, 5]
plt.rcParams['figure.facecolor'] = '#252525'
ax2 = df.plot.bar(zorder=3, color=[colors])
ax2.set_xlabel('Food category', color='#CCCCCC')
ax2.set_ylabel('UK GHG emissions (kgCO2e/100g)', color='#CCCCCC')
ax2.set_title('UK CO2 Equivalent GHG Emissions')
custom2 = [ Line2D([0], [0], color=d['r'], lw=5),
            Line2D([0], [0], color=d['m'], lw=5),
           Line2D([0], [0], color=d['b'], lw=5),
            Line2D([0], [0], color=d['g'], lw=5),
            Line2D([0], [0], color=d['y'], lw=5)]
ax2.legend(custom2, ['Meats & Eggs', 'Dairy', 'Grains & Starchy Foods', 'Nuts, Fruits & Vegetables', 'Soy'])
for tick in ax2.get_xticklabels():
    tick.set_rotation(45)
    tick.set_ha('right')
plt.savefig('plots/food_emissions.png', dpi=150)

### Translate Cesar's recipes to generic food groups
We use the healthy, nutritious vegan recipes Cesar made as an example vegan diet, so that we can compare it to an example meat diet obtained from the UK government [3].

In [ ]:
# Meal plan 1, converted to my food groups, with values in grams. See food_data.ipynb
m1 = [
    {
        'Starchy foods': 86,
        'Soy-based high-protein foods/drinks': 243,
        'Fruit and vegetables': 250,
        'Nuts and seeds': 30,
        'Nuts and seeds': 14
    },
    {
        'Starchy foods': 306,
        'Starchy foods': 240,
        'Soy-based high-protein foods/drinks': 166,
        'Fruit and vegetables': 68
    },
    {
        'Starchy foods': 242,
        'Fruit and vegetables': 88},
    {
        'Fruit and vegetables': 118,
        'Nuts and seeds': 60},
]

In [ ]:
# Meal plan 2, converted to my food groups, with values in grams. See food_data.ipynb
m2 = [
    {
        'Fruit and vegetables': 96,
        'Nuts and seeds': 60,
        'Fruit and vegetables': 180,
        'Fruit and vegetables': 96,
        'Fruit and vegetables': 140,
    },
    {
        'Starchy foods': 306,
        'Soy-based high-protein foods/drinks': 162,
        'Starchy foods': 172,
        'Fruit and vegetables': 84
    },
    {
        'Fruit and vegetables': 192,
    },
    {
        'Fruit and vegetables': 97,
        'Nuts and seeds': 30
    }
]

In [ ]:
# Meal plan 3, converted to my food groups, with values in grams. See food_data.ipynb
m3 = [
    {
        'Starchy foods': 370,
        'Soy-based high-protein foods/drinks': 162,
        'Starchy foods': 172,
        'Fruit and vegetables': 68,
        'Fruit and vegetables': 40,
        'Fruit and vegetables': 38,
    },
    {
        'Fruit and vegetables': 228,
        'Fruit and vegetables': 118,
        'Nuts and seeds': 32
    },
    {
        'Starchy foods': 158,
        'Starchy foods': 211,
        'Fruit and vegetables': 85
    },
    {
        'Nuts and seeds': 60,
        'Fruit and vegetables': 131
    },
]

In [ ]:
# create dataframe from vegan meals
food_groups = {}
for m in [m1, m2, m3]:
    for d_ in m:
        for k,v in d_.items():
            if k in food_groups:
                food_groups[k] += v
            else:
                food_groups[k] = v
df1 = pd.Series(food_groups, name='Vegan')
df1 = pd.DataFrame(df1)
sums = df1['Vegan'].sum()
df1['Vegan'] *= 100 / sums

### Compare example vegan diet and example omnivorous diet
We manually scrape a table of food group proportions recommended by the UK government [3] (since our other environmental data is from the UK). We clean that dataframe, and merge it with our example vegan diet dataframe. We then plot them side by side using pie charts.

In [ ]:
# table extracted from [3]
# wrangle table. Need to rename rows without commas
df = pd.read_csv('data/contribution_to_diet_by_food_group.csv', index_col='Food group')
df = df.merge(df1, how='outer', left_index=True, right_index=True)
df = df.rename(index={'Fish eggs meat and other proteins': 'Fish, eggs, meat and other proteins'})
df = df.iloc[[3,4,5,6,0,1,2]]

df.to_csv('data/contribution_to_diet_by_food_group_processed.csv')

# plot and format data
plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['figure.facecolor'] = '#252525'
props = dict(
    title='Contribution to Total Diet by Food Group',
    subplots=True,
    legend=False
)

colors = 'ggybmrg'
colors = [d[c] for c in list(colors)]


ax1, ax2 = df.plot.pie(**props, colors=colors, wedgeprops=dict(edgecolor='w'))

ax1.set_title('UK gov recommended omnivorous diet', color='#CCCCCC')
ax2.set_title('Generated healthy vegan diet', color='#CCCCCC')
ax1.set_ylabel('')
ax2.set_ylabel('')

custom2 = [ Line2D([0], [0], color=d['r'], lw=5),
            Line2D([0], [0], color=d['m'], lw=5),
           Line2D([0], [0], color=d['b'], lw=5),
            Line2D([0], [0], color=d['g'], lw=5),
            Line2D([0], [0], color=d['y'], lw=5)]
plt.legend(custom2, ['Meats & Eggs', 'Dairy', 'Grains & Starchy Foods', 'Nuts, Fruits & Vegetables', 'Soy'],
          bbox_to_anchor=(0,.45))
plt.savefig('plots/contribution_to_diet_by_food_group_processed.png', dpi=150)

### References
1. Scarborough, P., Appleby, P.N., Mizdrak, A. et al. Climatic Change (2014) 125: 179. https://doi.org/10.1007/s10584-014-1169-1
2. Vieux, F. et al. Greenhouse gas emissions of self-selected individual diets in France: Changing the diet structure or consuming less? (2012) 75: 91. https://doi.org/10.1016/j.ecolecon.2012.01.003
3. From Plate to Guide: What, why and how for the eatwell model. https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/579388/eatwell_model_guide_report.pdf